In [ ]:
from argparse import Namespace # 管理所有超参数
from collections import Counter # 统计词语数量
import string # 标点符号调用
import re # 正则表达式
import os # 生成文件路径
import json # 保存模型为JSON格式
import numpy as np # 数据处理
import pandas as pd # 文本处理
import torch # 调用PyTorch库
import torch.nn as nn # 调用神经网络层
import torch.nn.functional as F # 调用激活函数
import torch.optim as optim # 调用优化器
from torch.utils.data import Dataset, DataLoader # 调用批生成器

In [ ]:
args = Namespace(
    news_csv="20200913-Top10-clean", # 数据集
    vectorizer_file="news_folder/vectorizer.json", # 向量化器保存的位置
    model_state_file="news_folder/model.pth", # 模型保存的位置
    predicted_file="news_folder/predicted.csv", # 预测文件的位置
    word_embedding_size=100, # 词向量维度大小
    rnn_hidden_size=64, # RNN的隐藏层大小
    num_epochs=20, # 模型训练轮数
    learning_rate=1e-3, # 学习率
    batch_size=64, # 批的大小
    seed=1337, # 设置种子
    early_stopping_criteria=3, # 超过未优化次数将停止训练
    sampling=0.05, # 用CPU训练时取样5%数据用于训练
    cutoff=2, # 设置词典中词的最小频率
    dropout=0.5 # dropout的概率
)

In [ ]:
class Vocabulary(object):
    
    '''创建一个词典类来管理数据集中每个词和对应索引的关系'''
    
    def __init__(self, token_to_idx={}):
        '''
        Args:
            token_to_idx: 载入预先生成好的词典，若没有会自动生成空词典
        '''
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token for token, idx in self._token_to_idx.items()}
    
    # 向双向词典中加入令牌，并返回令牌在词典中所在的索引，若令牌已存在，直接返回索引
    def add_token(self, token):
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index

    # 查找令牌在词典中的对应索引
    def lookup_token(self, token):
        return self._token_to_idx[token]

    # 查找索引在词典中对应的令牌，若索引不存在将报错
    def lookup_index(self, index):
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]
    
    # 生成序列化信息，方便使用JSON保存初始化信息
    def to_serializable(self):
        return {'token_to_idx': self._token_to_idx}

    # 通过使用contents(序列化后的初始化信息)重建实例
    @classmethod
    def from_serializable(cls, contents):
        return cls(**contents)

    # Print打印实例的输出结果
    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    # 定义实例的长度信息为词典的长度
    def __len__(self):
        return len(self._token_to_idx)
    

In [ ]:
class SequenceVocabulary(Vocabulary):
    
    '''创建一个词典类来管理数据集中每个词和对应索引的关系'''

    def __init__(self, token_to_idx={}, unk_token='<UNK>', mask_token='<MASK>',
                 begin_token='<BEGIN>', end_token='<END>'):
        '''
        Args:
            token_to_idx: 载入预先生成好的词典，若没有会自动生成空词典
            unk_token，mask_token，begin_token, end_token: 文本中的特殊令牌
        '''
        super(SequenceVocabulary, self).__init__(token_to_idx)
        
        # 保存特殊的令牌
        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_token = begin_token
        self._end_token = end_token
        
        # 将特殊令牌添加到词典中，并保存对应的索引
        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_index = self.add_token(self._begin_token)
        self.end_index = self.add_token(self._end_token)

    # 查找令牌在词典中对应的索引，如果令牌不存在，则返回UNK索引
    def lookup_token(self, token):
        return self._token_to_idx.get(token, self.unk_index)
    
    # 生成序列化信息，方便使用JSON保存初始化信息
    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_token': self._begin_token,
                         'end_token': self._end_token})
        return contents
    

In [ ]:
class NewsVectorizer(object):
    
    '''创建一个向量化器类将文本句子转换为句子索引向量'''
    
    def __init__(self, word_vocab, label_vocab):
        '''Args:
               word_vocab: 包含数据集中所有文本的词典
               label_vocab: 包含数据集中所有标签的词典
        '''
        self.word_vocab = word_vocab
        self.label_vocab = label_vocab
        
    # 向量化文本句子，将句子中的每个单词用索引表示，生成句子索引向量
    def vectorize(self, news_title, vector_length=-1):
        indices = [self.word_vocab.begin_index]
        indices.extend([self.word_vocab.lookup_token(token) 
                       for token in news_title.split(' ')])
        indices.append(self.word_vocab.end_index)
        if vector_length == -1:
            vector_length = len(indices)
        title_vector = np.zeros(vector_length, dtype=np.int64)
        title_vector[:len(indices)] = indices
        title_vector[len(indices):] = self.word_vocab.mask_index
        return title_vector, len(indices)
    
    # 通过新闻数据集创建一个向量化器
    @classmethod
    def from_dataframe(cls, news_df, cutoff=args.cutoff):
        label_vocab = Vocabulary()
        for label in sorted(set(news_df['一级类目'])):
            label_vocab.add_token(label)
        word_counts = Counter()
        for title in news_df['新闻标题']:
            for token in title.split(" "):
                if token not in string.punctuation:
                    word_counts[token] += 1 
        word_vocab = SequenceVocabulary()
        for word, word_count in word_counts.items():
            if word_count >= cutoff:
                word_vocab.add_token(word)
        return cls(word_vocab, label_vocab)

    # 生成序列化信息，方便使用JSON保存初始化信息
    def to_serializable(self):
        return {'word_vocab': self.word_vocab.to_serializable(), 
                'label_vocab': self.label_vocab.to_serializable()}
    
    # 通过使用contents(序列化后的初始化信息)重建实例
    @classmethod
    def from_serializable(cls, contents):
        word_vocab = SequenceVocabulary.from_serializable(contents['word_vocab'])
        label_vocab = Vocabulary.from_serializable(contents['label_vocab'])
        return cls(word_vocab, label_vocab)
    

In [ ]:
class NewsDataset(Dataset):
    
    '''创建一个新闻数据类来对数据进行向量化和分组'''
    
    def __init__(self, news_df, vectorizer):
        '''
        Args:
            news_df: 新闻数据集
            vectorizer: 由训练集生成的向量化器
        '''
        self.news_df = news_df
        self._vectorizer = vectorizer
        # 计算数据集中最长文本的长度，用于之后的句子向量化
        self._max_seq_length = max(map(len, self.news_df["新闻标题"])) + 2
        self.train_df = self.news_df.iloc[0:int(len(self.news_df)*0.7)]
        self.train_size = len(self.train_df)
        self.val_df = self.news_df.iloc[int(len(self.news_df)*0.7):int(len(self.news_df)*0.85)]
        self.val_size = len(self.val_df)
        self.test_df = self.news_df.iloc[int(len(self.news_df)*0.85):]
        self.test_size = len(self.test_df)
        # 将数据集分划后保存在dict中，通过set_split调取需要使用的数据集
        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.val_size),
                             'test': (self.test_df, self.test_size)}
        self.set_split('train')
        
        # 由于一级类目的样本不平衡而计算的样本权重，用于softmax加权
        class_counts = self.train_df['一级类目'].value_counts().to_dict()
        def sort_key(item):
            return self._vectorizer.label_vocab.lookup_token(item[0])
        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)
        
    # 根据情况选择当前要使用的数据集，默认使用训练集
    def set_split(self, split="train"):
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
         
    # 定义数据集的长度，用于DataLoader的batch数量计算
    def __len__(self):
        return self._target_size
        
    # 定义数据集的输出，用于DataLoader的batch数据生成
    def __getitem__(self, index):
        row = self._target_df.iloc[index]
        news_vector, vec_length = \
        self._vectorizer.vectorize(row["新闻标题"], self._max_seq_length)
        label_index = \
        self._vectorizer.label_vocab.lookup_token(row["一级类目"])
        return {'x_data': news_vector, 
                'y_target': label_index, 
                'x_length': vec_length}
        
    # 用于之后的vectorizer提取使用
    def get_vectorizer(self):
        return self._vectorizer
        
    # 通过新闻数据集创建数据集实例
    @classmethod
    def load_dataset_and_make_vectorizer(cls, news_csv, sampling):
        news_df = pd.read_csv(news_csv)
        news_df = news_df.iloc[0:int(len(news_df)*sampling)]
        train_df = news_df.iloc[0:int(len(news_df)*0.7)]
        return cls(news_df, NewsVectorizer.from_dataframe(train_df))
        
    # 通过数据集以及保存好的向量化器来创建数据集实例
    @classmethod
    def load_dataset_and_load_vectorizer(cls, news_csv, vectorizer_filepath, sampling):
        news_df = pd.read_csv(news_csv)
        news_df = news_df.iloc[0:int(len(news_df)*sampling)]
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(news_df, vectorizer)
        
    # 从JSON文件中加载保存好的向量化器
    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        with open(vectorizer_filepath) as fp:
            return NewsVectorizer.from_serializable(json.load(fp))
        
    # 将向量化器保存到JSON文件中
    def save_vectorizer(self, vectorizer_filepath):
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)
                    

In [ ]:
class ElmanRNN(nn.Module):
    
    '''使用RNNCell创建一个RNN层'''
    
    def __init__(self, input_size, hidden_size, batch_first=False):
        '''
        Args:
            input_size: RNN输入数据的维度
            hidden_size: RNN的隐藏层大小
            batch_first: batch是否为数据集的第0维
        '''
        super(ElmanRNN, self).__init__()
        self.rnn_cell = nn.RNNCell(input_size, hidden_size)
        self.batch_first = batch_first
        self.hidden_size = hidden_size
        
    # 初始化隐藏层数值
    def _initial_hidden(self, batch_size):
        return torch.zeros((batch_size, self.hidden_size))
        
    # 输入数据批，返回每一个时间步长上的隐藏层数值
    def forward(self, x_in, initial_hidden=None):
        if self.batch_first:
            batch_size, seq_size, feat_size = x_in.size()
            x_in = x_in.permute(1, 0, 2)
        else:
            seq_size, batch_size, feat_size = x_in.size()
        hiddens = []
        if initial_hidden is None:
            initial_hidden = self._initial_hidden(batch_size)
        hidden_t = initial_hidden
        for t in range(seq_size):
            hidden_t = self.rnn_cell(x_in[t], hidden_t)
            hiddens.append(hidden_t)
        hiddens = torch.stack(hiddens)
        if self.batch_first:
            hiddens = hiddens.permute(1, 0, 2)
        return hiddens


In [ ]:
class NewsClassifier(nn.Module):
    
    '''创建新闻标题分类器'''
    
    def __init__(self, num_embeddings, embedding_dim, padding_idx,
                 rnn_hidden_size, num_classes, batch_first):
        '''
        Args:
            num_embeddings: 词嵌入矩阵的行数，等于词典单词的数量
            embedding_dim: 词嵌入矩阵的维度，人为规定大小
            padding_idx: 将某个index对应的令牌作为padding对象
            rnn_hidden_size: RNN的隐藏层大小
            num_classes: 输出层的大小
            batch_first: batch是否为数据集的第0维
        '''
        super(NewsClassifier, self).__init__()
        self.emb = nn.Embedding(num_embeddings=num_embeddings,
                                embedding_dim=embedding_dim,
                                padding_idx=padding_idx)
        self.rnn = ElmanRNN(input_size=embedding_dim,
                            hidden_size=rnn_hidden_size,
                            batch_first=batch_first)
        self.fc1 = nn.Linear(in_features=rnn_hidden_size,
                             out_features=rnn_hidden_size)
        self.fc2 = nn.Linear(in_features=rnn_hidden_size,
                             out_features=num_classes)
    
    # 输入数据批，返回批的最后一个时间步长上的隐藏层数值
    def forward(self, x_in, x_lengths=None, apply_softmax=False):
        x_embedded = self.emb(x_in)
        y_out = self.rnn(x_embedded)
        if x_lengths is not None:
            x_lengths = x_lengths.long().detach().cpu().numpy() - 1
            out = []
            for batch_index, column_index in enumerate(x_lengths):
                out.append(y_out[batch_index, column_index])
            y_out = torch.stack(out)
        else:
            y_out = y_out[:, -1, :]
        y_out = F.relu(self.fc1(F.dropout(y_out, args.dropout)))
        y_out = self.fc2(F.dropout(y_out, args.dropout))
        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)
        return y_out  
        

In [ ]:
# 用来跟踪模型的训练过程以及控制训练状态
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}


In [ ]:
# 每轮训练结束将更新一次训练状态
def update_train_state(args, model, train_state):
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False
    elif train_state['epoch_index'] >= 1:
        loss_pre_t, loss_t = train_state['val_loss'][-2:]
        if loss_t >= loss_pre_t:
            train_state['early_stopping_step'] += 1
        else:
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                train_state['early_stopping_best_val'] = loss_t
            train_state['early_stopping_step'] = 0
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria
    return train_state


In [ ]:
# 计算模型的准确度
def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100


In [ ]:
# 训练模型，验证模型，测试模型，保存模型
def train_model(args, dataset, classifier, optimizer, loss_func, scheduler):
    train_state = make_train_state(args)
    try:
        for epoch_index in range(args.num_epochs):
            train_state['epoch_index'] = epoch_index
            # 训连模型并将每轮训练结果用于更新状态
            dataset.set_split('train')
            batch_generator = DataLoader(dataset=dataset, batch_size=args.batch_size)
            running_loss = 0.0
            running_acc = 0.0
            classifier.train()
            for batch_index, batch_dict in enumerate(batch_generator):
                y_pred = classifier(x_in=batch_dict['x_data'], x_lengths=batch_dict['x_length'])
                loss = loss_func(y_pred, batch_dict['y_target'])
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                running_loss += (loss.item() - running_loss) / (batch_index + 1)
                acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
                running_acc += (acc_t - running_acc) / (batch_index + 1)
            train_state['train_loss'].append(running_loss)
            train_state['train_acc'].append(running_acc)
            # 验证模型并将验证结果用于更新状态
            dataset.set_split('val')
            batch_generator = DataLoader(dataset=dataset, batch_size=args.batch_size)
            running_loss = 0.0
            running_acc = 0.0
            classifier.eval()
            for batch_index, batch_dict in enumerate(batch_generator):
                y_pred = classifier(x_in=batch_dict['x_data'], x_lengths=batch_dict['x_length'])
                loss = loss_func(y_pred, batch_dict['y_target'])
                running_loss += (loss.item() - running_loss) / (batch_index + 1)
                acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
                running_acc += (acc_t - running_acc) / (batch_index + 1)
            train_state['val_loss'].append(running_loss)
            train_state['val_acc'].append(running_acc)
            # 更新训练状态
            train_state = update_train_state(args=args, model=classifier, train_state=train_state)
            # 更新学习率
            scheduler.step(train_state['val_loss'][-1])
            # 打印每轮训练的结果
            print("Epoch: {} / {} -- Train Accuracy: {:.3f}, Val Accuracy: {:.3f}".format(
                train_state['epoch_index']+1, args.num_epochs, train_state['train_acc'][-1], 
                train_state['val_acc'][-1]))
            # 判断是否提前结速训练
            if train_state['stop_early']:
                print('Early Stop Training!')
                break
        # 使用测试集测试训练好的模型，更新状态中的测试结果
        classifier.load_state_dict(torch.load(train_state['model_filename']))
        loss_func = nn.CrossEntropyLoss(dataset.class_weights)
        dataset.set_split('test')
        batch_generator = DataLoader(dataset=dataset, batch_size=args.batch_size)
        running_loss = 0.0
        running_acc = 0.0
        classifier.eval()
        for batch_index, batch_dict in enumerate(batch_generator):
            y_pred = classifier(batch_dict['x_data'], x_lengths=batch_dict['x_length'])
            loss = loss_func(y_pred, batch_dict['y_target'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
        train_state['test_loss'] = running_loss
        train_state['test_acc'] = running_acc
        print("Test Accuracy: {:.3f}".format(train_state['test_acc']))
    except KeyboardInterrupt:
        print("Exiting loop")
        

In [ ]:
# 准备训练模型需要的所有工具
def prepare_training_process(args):
    # 设置概率种子
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    # 初始化保存文件夹
    if not os.path.exists('news_folder/'):
        os.makedirs('news_folder/')
    # 准备数据集
    if os.path.exists(args.vectorizer_file):
        dataset = NewsDataset.load_dataset_and_load_vectorizer(
            args.news_csv, args.vectorizer_file, args.sampling)
    else:
        dataset = NewsDataset.load_dataset_and_make_vectorizer(args.news_csv, args.sampling)
        dataset.save_vectorizer(args.vectorizer_file)
    # 准备向量化器
    vectorizer = dataset.get_vectorizer()
    # 准备分类器
    classifier = NewsClassifier(
                 num_embeddings=len(vectorizer.word_vocab),
                 embedding_dim=args.word_embedding_size,
                 padding_idx=vectorizer.word_vocab.mask_index,
                 rnn_hidden_size=args.rnn_hidden_size,
                 num_classes=len(vectorizer.label_vocab),
                 batch_first=True)
    # 准备优化器
    optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
    # 准备损失函数
    loss_func = nn.CrossEntropyLoss(dataset.class_weights)
    # 准备学习率调整器
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.5, patience=1)
    return dataset, vectorizer, classifier, optimizer, loss_func, scheduler


In [ ]:
# 训练分类器模型，并保存到文件夹中
dataset, vectorizer, classifier, optimizer, loss_func, scheduler = prepare_training_process(args)
train_model(args, dataset, classifier, optimizer, loss_func, scheduler)

In [ ]:
def predict_label(classifier, vectorizer):
    news_df = pd.read_csv(args.news_csv)
    news_df = news_df.iloc[0:int(len(news_df)*0.01)]
    predicted_news_df = []
    for news_title in news_df['新闻标题']:
        vectorized_news, vec_length = vectorizer.vectorize(news_title)
        vectorized_news = torch.tensor(vectorized_news).unsqueeze(dim=0)
        vec_length = torch.tensor([vec_length], dtype=torch.int64)
        result = classifier(vectorized_news, vec_length, apply_softmax=True)
        probability_values, indices = result.max(dim=1)
        index = indices.item()
        prob_value = probability_values.item()
        predicted_label = vectorizer.label_vocab.lookup_index(index)
        predicted_result = {'news_title': news_title, 'label': predicted_label, 'probability': prob_value}
        predicted_news_df.append(predicted_result)
    output_df = pd.DataFrame(predicted_news_df)
    output_df.to_csv(args.predicted_file, index=False)
    return output_df

In [ ]:
output_df = predict_label(classifier, vectorizer)